In [3]:
from torch import nn
import wandb
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## Data Normalization

## Hyperparameter Tuning

## Model Training

## Model Definitions

In [ ]:


# simple neural network

class SimpleNN(nn.Module):
	def __init__(self, input_size, num_classes):
		super(SimpleNN, self).__init__()
		self.fc1 = nn.Linear(input_size, int(input_size/2))
		self.relu = nn.ReLU()
		self.fc2 = nn.Linear(int(input_size/2), int(input_size/4))
		self.fc3 = nn.Linear(int(input_size/4), num_classes)

	def forward(self, x):
		out = self.fc1(x)
		out = self.relu(out)
		out = self.fc2(out)
		out = self.relu(out)
		out = self.fc3(out)
		return out


In [ ]:
# convolutional neural network
# variable input length

class ConvNN(nn.Module):

	def __init__(self, num_classes):
		super(ConvNN, self).__init__()
		self.layer1 = nn.Sequential(
			nn.Conv1d(1, 16, kernel_size=5, stride=1, padding=2),
			nn.ReLU(),
			nn.MaxPool2d(kernel_size=2, stride=2))
		self.layer2 = nn.Sequential(
			nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2),
			nn.ReLU(),
			nn.MaxPool2d(kernel_size=2, stride=2))
		self.fc = nn.Linear(7*7*32, num_classes)

In [ ]:
# model training
def train_model(model, criterion, optimizer, num_epochs, train_loader, test_loader, wandb):
	total_step = len(train_loader)
	for epoch in range(num_epochs):
		for i, (images, labels) in enumerate(train_loader):
			images = images.to(device)
			labels = labels.to(device)
			outputs = model(images)
			loss = criterion(outputs, labels)
			optimizer.zero_grad()
			loss.backward()
			optimizer.step()
			if (i+1) % 10 == 0:
				print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'
					   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))
				wandb.log({'training_loss': loss.item()})
		val_loss = 0
		with torch.no_grad():
			for i, (images, labels) in enumerate(test_loader):
				images = images.to(device)
				labels = labels.to(device)
				outputs = model(images)
				val_loss += criterion(outputs, labels)
		print('Validation loss: {:.4f}'.format(val_loss))
		wandb.log({'validation_loss': val_loss})
		return model

In [ ]:
input_size = x
lr = 0.001
training_size = y
testing_size = z
model = SimpleNN(1000, 8).to(device)
batch_size = t
num_epochs = 5
architecture = 'FNN' if isinstance(model, SimpleNN) else 'CNN'
print(architecture)

wandb.init(
    # set the wandb project where this run will be logged
    project="ml4Oxi",
    # track hyperparameters and run metadata
    config={
    "learning_rate": lr,
    "architecture": architecture,
    "dataset": "MP_only",
    "epochs": num_epochs,
    }
)

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

model = train_model(model, criterion, optimizer, 5, train_loader, test_loader, wandb)


## Model Evaluation

## Performance Analysis